decision tree - yes/no classification tree
root - branch - leaf

In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('scoring_raw.csv')
df.head()

,age,months_on_file,annual_income,monthly_income,total_credit_limit,current_balance,num_open_accounts,num_closed_accounts,credit_utilization,avg_payment_delay_days,...,marital_status,residence_type,device_type,kyc_level,has_mobile_wallet,primary_bank,industry,card_type,default_flag,risk_score
0,38.483896,60.130698,1608.520603,134.043384,160.643860,25.287924,2.0,0,0.157416,0.417616,...,widowed,own,android,basic,yes,Ipoteka,services,credit,0,224
1,49.601710,101.801386,82249.760666,6854.146722,57715.606073,38215.483488,9.0,3,0.662134,7.926312,...,married,family,android,enhanced,no,UZCARD,services,credit,1,896
2,45.995639,91.053755,30567.582207,2547.298517,14314.990727,8174.391542,8.0,2,0.571037,5.740914,...,single,own,android,basic,yes,Ipoteka,construction,credit,0,807
3,52.479830,113.901075,243116.557392,20259.713116,340627.583117,274741.766249,10.0,5,0.806575,11.512414,...,widowed,dorm,ios,enhanced,yes,UZCARD,services,prepaid,1,931
4,41.956295,81.028236,14695.400359,1224.616697,2272.975873,805.934938,5.0,0,0.354573,2.308756,...,widowed,own,feature_phone,basic,no,Qishloq,retail,credit,1,670


In [3]:
df.shape

(6000, 50)

In [4]:
df.columns

Index(['age', 'months_on_file', 'annual_income', 'monthly_income',
       'total_credit_limit', 'current_balance', 'num_open_accounts',
       'num_closed_accounts', 'credit_utilization', 'avg_payment_delay_days',
       'num_delinquencies_12m', 'num_hard_inquiries_6m',
       'num_soft_inquiries_6m', 'home_value_estimate', 'loan_amount_existing',
       'loan_count_existing', 'employment_years', 'dependents',
       'education_score', 'savings_balance', 'checking_balance',
       'debt_to_income', 'rent_or_mortgage', 'late_fee_paid_12m',
       'last_payment_days_ago', 'max_past_due', 'avg_monthly_spend',
       'card_tenure_months', 'credit_age_months', 'pct_high_balance_accounts',
       'pct_on_time_payments', 'chargeoff_history_count', 'revolving_balance',
       'installment_balance', 'utilization_3m_avg', 'utilization_12m_avg',
       'merchant_category_diversity', 'income_volatility_index', 'region',
       'employment_type', 'marital_status', 'residence_type', 'device_type',
 

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   age                          6000 non-null   float64
 1   months_on_file               6000 non-null   float64
 2   annual_income                6000 non-null   float64
 3   monthly_income               6000 non-null   float64
 4   total_credit_limit           6000 non-null   float64
 5   current_balance              6000 non-null   float64
 6   num_open_accounts            6000 non-null   float64
 7   num_closed_accounts          6000 non-null   int64  
 8   credit_utilization           6000 non-null   float64
 9   avg_payment_delay_days       6000 non-null   float64
 10  num_delinquencies_12m        6000 non-null   int64  
 11  num_hard_inquiries_6m        6000 non-null   int64  
 12  num_soft_inquiries_6m        6000 non-null   int64  
 13  home_value_estimat

1. Random Forest

In [9]:
# split and one-hot encode categorical features
# X = df.drop('default_flag', axis=1)


In [10]:
# Random Forest pipeline + RandomizedSearchCV (AUC) — ready to run
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import randint
import time

df = pd.read_csv('scoring_raw.csv')


In [11]:
categorical_cols = [
    'region','employment_type','marital_status','residence_type',
    'device_type','kyc_level','has_mobile_wallet','primary_bank',
    'industry','card_type'
]

target = 'default_flag'

# numeric columns = all except categorical + target + risk_score (optional)
numeric_cols = [c for c in df.columns if c not in categorical_cols + [target, 'risk_score']]

# 2) train/test split (stratify by target since credit datasets often imbalanced)
X = df[numeric_cols + categorical_cols]
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

In [14]:
# 3) preprocessing: OneHotEncoder (dense) for categorical, passthrough for numeric
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # dense 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', ohe, categorical_cols),
    ],
    remainder='drop',
    verbose_feature_names_out=False  # simpler names if available in your sklearn
)

preprocessor

,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,None
,sparse_output,False


In [17]:
# 4) pipeline with RandomForest
rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', rf)
])

pipeline

,steps,"[('preprocessor', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False


In [18]:
# 5) hyperparameter distributions for RandomizedSearchCV (starter small search)
param_dist = {
    'clf__n_estimators': [100, 300, 500],
    'clf__max_depth': [4, 8, 16, None],
    'clf__max_features': ['sqrt', 'log2', 0.3],
    'clf__min_samples_leaf': [1, 2, 5],
    'clf__min_samples_split': [2, 5, 10]
}


In [19]:
# 6) RandomizedSearchCV setup
n_iter = 40
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=n_iter,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    return_train_score=False
)

In [20]:
# 7) fit (time it)
start = time.time()
random_search.fit(X_train, y_train)
end = time.time()
print(f"\nRandomizedSearchCV fit time: {end - start:.1f} seconds\n")

Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=5, clf__min_samples_split=10, clf__n_estimators=100; total time=   1.7s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=100; total time=   1.4s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=5, clf__min_samples_split=10, clf__n_estimators=100; total time=   1.5s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=5, clf__min_samples_split=10, clf__n_estimators=100; total time=   1.8s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=5, clf__min_samples_split=10, clf__n_estimators=100; total time=   2.0s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=100; total time=   1.8s
[CV] END clf__max_depth=8, clf__max_features=log2, clf__min_samples_le

In [21]:
# 8) report best params and CV score
print("Best parameters (from RandomizedSearchCV):")
print(random_search.best_params_)
print(f"Best cross-validated AUC: {random_search.best_score_:.4f}")

Best parameters (from RandomizedSearchCV):
{'clf__n_estimators': 500, 'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_features': 0.3, 'clf__max_depth': 8}
Best cross-validated AUC: 0.8441


In [22]:
# 9) Evaluate on hold-out test set
best_pipeline = random_search.best_estimator_
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test set AUC: {test_auc:.4f}")

Test set AUC: 0.8429


In [23]:
# 10) Get feature names and show top feature importances
# - numeric feature names are numeric_cols
# - ohe feature names from the trained OneHotEncoder inside the ColumnTransformer
# Access fitted preprocessor and OHE
fitted_preprocessor = best_pipeline.named_steps['preprocessor']
# numeric names
num_names = numeric_cols

# get the fitted OneHotEncoder object
fitted_ohe = fitted_preprocessor.named_transformers_['cat']
# get OHE columns (requires sklearn ≥ 1.0). If not available, fallback below.
try:
    ohe_feature_names = fitted_ohe.get_feature_names_out(categorical_cols)
except Exception:
    # fallback: build names from categories_
    ohe_feature_names = []
    for col, cats in zip(categorical_cols, fitted_ohe.categories_):
        ohe_feature_names.extend([f"{col}__{val}" for val in cats])

feature_names = np.array(list(num_names) + list(ohe_feature_names))



In [24]:
# feature importances from the RandomForest inside pipeline
importances = best_pipeline.named_steps['clf'].feature_importances_
# ensure lengths match
if len(importances) == len(feature_names):
    fi = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    print("\nTop 20 feature importances:")
    print(fi.head(20).to_string(index=False))
else:
    print("Warning: feature importances length doesn't match feature names length.")
    print(f"len(importances)={len(importances)}, len(feature_names)={len(feature_names)}")


Top 20 feature importances:
                    feature  importance
       loan_amount_existing    0.111094
              annual_income    0.107283
             monthly_income    0.095619
             debt_to_income    0.066346
      last_payment_days_ago    0.062399
            savings_balance    0.051467
     avg_payment_delay_days    0.050637
               max_past_due    0.040171
           checking_balance    0.034474
        utilization_12m_avg    0.033127
         card_tenure_months    0.027365
                        age    0.024717
merchant_category_diversity    0.022501
          avg_monthly_spend    0.022483
             months_on_file    0.019487
        home_value_estimate    0.018767
         utilization_3m_avg    0.018204
  pct_high_balance_accounts    0.016623
           rent_or_mortgage    0.014258
            education_score    0.013152


In [25]:
# Optional: save the best pipeline for later
import joblib
joblib.dump(best_pipeline, 'best_rf_pipeline.joblib')
print("\nSaved best pipeline to 'best_rf_pipeline.joblib'.")


Saved best pipeline to 'best_rf_pipeline.joblib'.


In [ ]:
# Random Forest pipeline + RandomizedSearchCV (AUC) — ready to run
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from scipy.stats import randint
import time

# ---- replace with your df if needed ----
df = pd.read_csv('scoring_raw.csv')
# ----------------------------------------

# 1) feature lists from earlier conversation
categorical_cols = [
    'region','employment_type','marital_status','residence_type',
    'device_type','kyc_level','has_mobile_wallet','primary_bank',
    'industry','card_type'
]

target = 'default_flag'

# numeric columns = all except categorical + target + risk_score (optional)
numeric_cols = [c for c in df.columns if c not in categorical_cols + [target, 'risk_score']]

# 2) train/test split (stratify by target since credit datasets often imbalanced)
X = df[numeric_cols + categorical_cols]
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# 3) preprocessing: OneHotEncoder (dense) for categorical, passthrough for numeric
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)  # dense as requested

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', ohe, categorical_cols),
    ],
    remainder='drop',
    verbose_feature_names_out=False  # simpler names if available in your sklearn
)

# 4) pipeline with RandomForest
rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('clf', rf)
])

# 5) hyperparameter distributions for RandomizedSearchCV (starter small search)
param_dist = {
    'clf__n_estimators': [100, 300, 500],
    'clf__max_depth': [4, 8, 16, None],
    'clf__max_features': ['sqrt', 'log2', 0.3],
    'clf__min_samples_leaf': [1, 2, 5],
    'clf__min_samples_split': [2, 5, 10]
}

# 6) RandomizedSearchCV setup
n_iter = 40
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=n_iter,
    scoring='roc_auc',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    return_train_score=False
)

# 7) fit (time it)
start = time.time()
random_search.fit(X_train, y_train)
end = time.time()
print(f"\nRandomizedSearchCV fit time: {end - start:.1f} seconds\n")

# 8) report best params and CV score
print("Best parameters (from RandomizedSearchCV):")
print(random_search.best_params_)
print(f"Best cross-validated AUC: {random_search.best_score_:.4f}")

# 9) Evaluate on hold-out test set
best_pipeline = random_search.best_estimator_
y_pred_proba = best_pipeline.predict_proba(X_test)[:, 1]
test_auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test set AUC: {test_auc:.4f}")

# 10) Get feature names and show top feature importances
# Note: extracting feature names varies with sklearn versions; below is robust.
# - numeric feature names are numeric_cols
# - ohe feature names from the trained OneHotEncoder inside the ColumnTransformer

# Access fitted preprocessor and OHE
fitted_preprocessor = best_pipeline.named_steps['preprocessor']
# numeric names
num_names = numeric_cols

# get the fitted OneHotEncoder object
fitted_ohe = fitted_preprocessor.named_transformers_['cat']
# get OHE columns (requires sklearn ≥ 1.0). If not available, fallback below.
try:
    ohe_feature_names = fitted_ohe.get_feature_names_out(categorical_cols)
except Exception:
    # fallback: build names from categories_
    ohe_feature_names = []
    for col, cats in zip(categorical_cols, fitted_ohe.categories_):
        ohe_feature_names.extend([f"{col}__{val}" for val in cats])

feature_names = np.array(list(num_names) + list(ohe_feature_names))

# feature importances from the RandomForest inside pipeline
importances = best_pipeline.named_steps['clf'].feature_importances_
# ensure lengths match
if len(importances) == len(feature_names):
    fi = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    print("\nTop 20 feature importances:")
    print(fi.head(20).to_string(index=False))
else:
    print("Warning: feature importances length doesn't match feature names length.")
    print(f"len(importances)={len(importances)}, len(feature_names)={len(feature_names)}")

# Optional: save the best pipeline for later
import joblib
joblib.dump(best_pipeline, 'best_rf_pipeline.joblib')
print("\nSaved best pipeline to 'best_rf_pipeline.joblib'.")
